In [18]:
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, GRU, Embedding, Dropout, Activation, BatchNormalization, AveragePooling1D
from keras.layers import Bidirectional, GlobalMaxPool1D, concatenate
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from Attention import *
import h5py

In [3]:
data = pd.read_csv('../data/processed/train_sent_analysis.csv')

In [4]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,prob_hate_speech,prob_offensive_language
0,22256635,"Nonsense? kiss off, geek. what I said is true...",1,0,0,0,0,0,0.098280,0.207343
1,27450690,"""\n\n Please do not vandalize pages, as you di...",0,0,0,0,0,0,0.077788,0.239647
2,54037174,"""\n\n """"Points of interest"""" \n\nI removed the...",0,0,0,0,0,0,0.008337,0.339472
3,77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0,0.095393,0.184936
4,79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0,0.070083,0.226756


In [41]:
classes = data.columns[2:-2].values

In [43]:
train_comments = data['comment_text'].fillna('UNK')
labels = data.loc[:, classes].values
sent_features = data.loc[:, ['prob_hate_speech','prob_offensive_language']].values

In [8]:
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 100

In [9]:
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(train_comments)

In [10]:
train_sequences = tokenizer.texts_to_sequences(train_comments)
X_train = pad_sequences(train_sequences, maxlen = MAX_SEQUENCE_LENGTH)

In [44]:
y_train = data[classes].values

In [12]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 153188 unique tokens.


In [13]:
! ls ../embeddings

glove.840B.300d.txt  glove.840B.300d.zip


In [14]:
embeddings_index = {}
f = open('../embeddings/glove.840B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except:
        print("error reading word", word)
f.close()

print('Found %s word vectors.' % len(embeddings_index))

error reading word .
error reading word at
error reading word .
error reading word to
error reading word .
error reading word .
error reading word email
error reading word or
error reading word contact
error reading word Email
error reading word on
error reading word At
error reading word by
error reading word in
error reading word emailing
error reading word Contact
error reading word at
error reading word •
error reading word at
error reading word is
Found 2195884 word vectors.


In [19]:
########################################
print('Preparing embedding matrix')
nb_words = min(MAX_NB_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix
Null word embeddings: 68661


In [20]:
embedding_matrix.shape

(153189, 300)

In [95]:
embedding_layer = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix], trainable=False)

In [98]:
comment_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
sent_input = Input(shape=(2,), dtype='float32')
x = embedding_layer(comment_input)
x = Bidirectional(LSTM(300, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = Attention()(x)
x = concatenate([x, sent_input])
x = Dense(200, activation='relu')(x)
x = Dropout(0.3)(x)
# x = BatchNormalization()(x)
# x = Dense(50, activation='relu')(x)
# x = Dropout(0.3)(x)
# x = BatchNormalization()(x)
predictions = Dense(6, activation='sigmoid')(x)

In [99]:
model = Model(inputs=[comment_input, sent_input], outputs=predictions)

In [100]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [101]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 100, 300)     45956700    input_19[0][0]                   
__________________________________________________________________________________________________
bidirectional_10 (Bidirectional (None, 100, 600)     1442400     embedding_6[1][0]                
__________________________________________________________________________________________________
attention_10 (Attention)        (None, 600)          700         bidirectional_10[0][0]           
__________________________________________________________________________________________________
input_20 (

In [102]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_ckpt = ModelCheckpoint(filepath='../models/bstlm_attention.h5', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5)
callbacks = [early_stopping, model_ckpt, reduce_lr]

In [103]:
model.fit([X_train, sent_features], y_train, batch_size=256, epochs=50, validation_split=0.01, callbacks=callbacks)

Train on 94892 samples, validate on 959 samples
Epoch 1/50
94892/94892 [==============================] - 133s 1ms/step - loss: 0.0790 - acc: 0.9737 - val_loss: 0.0613 - val_acc: 0.9753
Epoch 2/50
94892/94892 [==============================] - 128s 1ms/step - loss: 0.0525 - acc: 0.9809 - val_loss: 0.0486 - val_acc: 0.9824
Epoch 3/50
94892/94892 [==============================] - 127s 1ms/step - loss: 0.0466 - acc: 0.9824 - val_loss: 0.0482 - val_acc: 0.9826
Epoch 4/50
94892/94892 [==============================] - 127s 1ms/step - loss: 0.0441 - acc: 0.9831 - val_loss: 0.0433 - val_acc: 0.9835
Epoch 5/50
94892/94892 [==============================] - 127s 1ms/step - loss: 0.0411 - acc: 0.9841 - val_loss: 0.0442 - val_acc: 0.9818
Epoch 6/50
94892/94892 [==============================] - 126s 1ms/step - loss: 0.0386 - acc: 0.9849 - val_loss: 0.0448 - val_acc: 0.9818
Epoch 7/50
94892/94892 [==============================] - 127s 1ms/step - loss: 0.0366 - acc: 0.9856 - val_loss: 0.0466 - va

In [25]:
test = pd.read_csv('../data/test.csv')

In [26]:
test_comments = test['comment_text'].fillna('UNK')
test_sequences = tokenizer.texts_to_sequences(test_comments)
X_test = pad_sequences(test_sequences, maxlen = MAX_SEQUENCE_LENGTH)

In [27]:
model.load_weights('../models/bstlm_attention.h5')

In [28]:
y_pred = model.predict(X_test)

In [30]:
sub = pd.read_csv('../data/sample_submission.csv')

In [37]:
sub.iloc[:, 1:] = y_pred

In [38]:
sub.to_csv('../submissions/bstlm_attention.csv', index=False)

In [40]:
sub.to_csv('bstlm_attention.csv', index=False)

In [39]:
pd.read_csv('../submissions/bstlm_attention.csv').head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,6044863,0.000402,0.000025,0.000044,4.169984e-06,0.000041,0.000004
1,6102620,0.000703,0.000010,0.000268,7.169110e-07,0.000042,0.000008
2,14563293,0.000136,0.000005,0.000077,4.495609e-07,0.000010,0.000003
3,21086297,0.001266,0.000038,0.000113,5.858633e-06,0.000108,0.000009
4,22982444,0.000594,0.000033,0.000063,6.175662e-06,0.000057,0.000006
